In [20]:
import tempfile
import os

import pandas as pd

import tensorflow as tf

In [21]:
URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00529/diabetes_data_upload.csv"

dataset = tf.keras.utils.get_file(
    "diabetes_data_upload.csv",
    URL)

40960/34682 [===================================] - 0s 10us/step


In [23]:
!pip install -q -U tensorflow_datasets

In [24]:
import tensorflow_datasets as tfds

In [25]:
CSV_COLUMNS =  ['age','gender','polyuria','polydipsia','sudden_weight_loss', 'weakness', 'polyphagia', 'genital_thrush', 'visual_blurring','itching','irritability', 'delayed_healing', 'partial_paresis', 'muscle_stiffness', 'alopecia', 'obesity', 'cls']


In [26]:
def train_input_fn():
  diabete = tf.data.experimental.make_csv_dataset(
      dataset,
      batch_size=32,
      header=True,
      column_names=CSV_COLUMNS,
      label_name="cls")
  diabete_batches = (
      diabete.cache().repeat().shuffle(100)
      .prefetch(tf.data.AUTOTUNE))
  return diabete_batches

In [27]:
def categorical_fc(name, values):
    """Helper function to wrap categorical feature by indicator column.

    Args:
        name: str, name of feature.
        values: list, list of strings of categorical values.
    Returns:
        Indicator column of categorical feature.
    """
    cat_column = tf.feature_column.categorical_column_with_vocabulary_list(
            key=name, vocabulary_list=values)

    return tf.feature_column.indicator_column(categorical_column=cat_column)

def create_feature_columns():
    """Creates dictionary of feature columns from inputs.

    Returns:
        Dictionary of feature columns.
    """
    feature_columns = {
        colname : tf.feature_column.numeric_column(key=colname)
           for colname in ["age"]
    }

    feature_columns["gender"] = categorical_fc("gender", ['Male', 'Female'])
    
    for colname in ['gender','polyuria','polydipsia','sudden_weight_loss', 'weakness', 'polyphagia', 'genital_thrush', 'visual_blurring','itching','irritability', 'delayed_healing', 'partial_paresis', 'muscle_stiffness', 'alopecia', 'obesity']:        
        feature_columns[colname] = categorical_fc(colname, ['Yes', 'No'])

    return feature_columns

In [28]:
fc = create_feature_columns()
for key,val in fc.items():
    exec(key + '=val')

In [29]:
from tensorflow import keras

model_dir = tempfile.mkdtemp()
model = tf.estimator.DNNClassifier(
    model_dir=model_dir,
    hidden_units=[36,12],
    feature_columns=[age,gender,polyuria,polydipsia,sudden_weight_loss,weakness,polyphagia,genital_thrush,visual_blurring,itching,irritability,delayed_healing,partial_paresis,muscle_stiffness,alopecia,obesity],
    n_classes=2,
    label_vocabulary=['Positive','Negative'],
    activation_fn='relu',
    optimizer=keras.optimizers.Adam(learning_rate=0.005)
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpmat3ny62', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [30]:
model = model.train(input_fn=train_input_fn, steps=8000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpmat3ny62/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 4.8283205, step = 0
INFO:tensorflow:global_step/sec: 149.307
INFO:tensorflow:loss = 0.3185474, step = 100 (0.673 sec)
INFO:tensorflow:global_step/sec: 193.233
INFO:tensorflow:loss = 0.293967, step = 200 (0.516 sec)
INFO:tensorflow:global_step/sec: 199.412
INFO:tensorflow:loss = 0.2986024, step = 300 (0.504 sec)
INFO:tensorflow:global_step/sec: 200.385
INFO:tensorflow:loss = 0.22637415, step = 400 (0.496 sec)
INFO:tensorflow:global_step/sec: 170.53
INFO:tensorflow:loss = 0.29818985, step = 500 (0.587 sec)
INFO:t

In [31]:
result = model.evaluate(train_input_fn, steps=10)

for key, value in result.items():
  print(key, ":", value)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-01-20T13:07:30Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpmat3ny62/model.ckpt-8000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/10]
INFO:tensorflow:Evaluation [2/10]
INFO:tensorflow:Evaluation [3/10]
INFO:tensorflow:Evaluation [4/10]
INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Inference Time : 1.20406s
INFO:tensorflow:Finished evaluation at 2021-01-20-13:07:32
INFO:tensorflow:Saving dict for global step 8000: accuracy = 0.984375, accuracy_baseline = 0.61875, auc = 0.9987374, auc_precision_recall = 0.99792963, average_loss = 0.03554583, global_step = 8000, label/mean = 0.38125, loss = 0.

In [32]:
for pred in model.predict(train_input_fn):
  for key, value in pred.items():
    print(key, ":", value)
  break

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpmat3ny62/model.ckpt-8000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
logits : [18.083221]
logistic : [1.]
probabilities : [1.40138265e-08 1.00000000e+00]
class_ids : [1]
classes : [b'Negative']
all_class_ids : [0 1]
all_classes : [b'Positive' b'Negative']
